In [3]:
import utility as ut
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from sklearn.neural_network import MLPClassifier
import xgboost as xgb

# Random Forest

In [4]:
# Make dataframe
df = ut.make_big_df()
df = ut.process(df)
df = df.dropna().reset_index(drop=True)

X = df.drop(columns=['Play Type']).to_numpy()
y = df['Play Type'].to_numpy()
df


C:\Users\calvi\anaconda3\envs\byu-football-play-predictions\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\calvi\anaconda3\envs\byu-football-play-predictions\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


,Offense Score,Defense Score,Drive Number,Play Number,Period,totalseconds,Offense Timeouts,Yards To Goal,Down,Distance,Play Type,L1 Yards Gained,L2 Yards Gained,L1 Play Type,L2 Play Type,L1 Down,L2 Down,L1 Distance,L2 Distance,point diff
0,0,0,1,4,1,840,3.0,71,3,6,1,4.0,0.0,3.0,1.0,2.0,1.0,10.0,10.0,0
1,0,0,1,5,1,833,3.0,61,1,10,1,10.0,4.0,1.0,3.0,3.0,2.0,6.0,10.0,0
2,0,0,1,6,1,827,3.0,61,2,10,1,0.0,10.0,1.0,1.0,1.0,3.0,10.0,6.0,0
3,0,0,1,7,1,821,3.0,61,3,10,1,0.0,0.0,1.0,1.0,2.0,1.0,10.0,10.0,0
4,0,0,1,8,1,814,3.0,61,4,10,2,0.0,0.0,1.0,1.0,3.0,2.0,10.0,10.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1550,35,20,22,1,4,162,-5.0,36,1,10,3,44.0,0.0,0.0,3.0,4.0,3.0,27.0,27.0,15
1551,35,20,22,3,4,155,-5.0,37,2,11,3,1.0,44.0,3.0,0.0,1.0,4.0,10.0,27.0,15
1552,35,20,22,5,4,150,-6.0,38,3,12,3,1.0,1.0,3.0,3.0,2.0,1.0,11.0,10.0,15
1553,35,20,22,7,4,96,-6.0,42,4,16,2,1.0,1.0,3.0,3.0,3.0,2.0,12.0,11.0,15


In [5]:
# Train random forest
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01)

clf = RandomForestClassifier()
rand_forest_cv = cross_val_score(clf, X_train, y_train, cv=20)
print("Cross-validation scores:", rand_forest_cv)
print("Mean CV accuracy:", rand_forest_cv.mean())
# Train on all data now that we have done CV
clf.fit(X, y)

y_pred = clf.predict(X)

# Calculate MAE, MSE, and RMSE
rand_forest_mae = mean_absolute_error(y, y_pred)
rand_forest_mse = mean_squared_error(y, y_pred)
rand_forest_rmse = np.sqrt(rand_forest_mse)

print(f"rand_forest_mae: {rand_forest_mae}")
print(f"rand_forest_mse: {rand_forest_mse}")
print(f"rand_forest_rmse: {rand_forest_rmse}")
# print(f"Training accuracy: {clf.score(X_train, y_train)}")
# print(f"Test accuracy: {clf.score(X_test, y_test)}")

Cross-validation scores: [0.66233766 0.63636364 0.66233766 0.5974026  0.67532468 0.71428571
 0.58441558 0.53246753 0.66233766 0.61038961 0.62337662 0.64935065
 0.71428571 0.61038961 0.71428571 0.55844156 0.58441558 0.63636364
 0.5974026  0.76315789]
Mean CV accuracy: 0.6394565960355434
rand_forest_mae: 0.0
rand_forest_mse: 0.0
rand_forest_rmse: 0.0


In [6]:
normalized_x = ut.normalize(X)
X_train, X_test, y_train, y_test = train_test_split(normalized_x, y, test_size=0.01)

norm_clf = RandomForestClassifier()
norm_clf.fit(X_train, y_train)

norm_rfst_cv = cross_val_score(clf, X_train, y_train, cv=20)
print("Cross-validation scores:", norm_rfst_cv)
print("Mean CV accuracy:", norm_rfst_cv.mean())

# Now train on all of the data
clf.fit(normalized_x, y)

y_pred = norm_clf.predict(normalized_x)

# Calculate MAE, MSE, and RMSE
norm_rand_forest_mae = mean_absolute_error(y, y_pred)
norm_rand_forest_mse = mean_squared_error(y, y_pred)
norm_rand_forest_rmse = np.sqrt(norm_rand_forest_mse)

print(f"norm_rand_forest_mae: {norm_rand_forest_mae}")
print(f"norm_rand_forest_mse: {norm_rand_forest_mse}")
print(f"norm_rand_foest_rmse: {norm_rand_forest_rmse}")

# print(f"Training accuracy: {clf.score(X_train, y_train)}")
# print(f"Test accuracy: {clf.score(X_test, y_test)}")

Cross-validation scores: [0.61038961 0.67532468 0.67532468 0.67532468 0.62337662 0.55844156
 0.62337662 0.61038961 0.63636364 0.61038961 0.74025974 0.66233766
 0.62337662 0.63636364 0.62337662 0.62337662 0.62337662 0.64935065
 0.7012987  0.65789474]
Mean CV accuracy: 0.6419856459330143
norm_rand_forest_mae: 0.0077170418006430866
norm_rand_forest_mse: 0.015434083601286173
norm_rand_foest_rmse: 0.12423398730333891


# XGBoost

In [10]:
df = ut.make_big_df()
df = ut.process(df)

X = df.drop(columns=['Play Type']).to_numpy()
y = df['Play Type'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

params = {
    'eta': 0.1,
    'objective': 'multi:softmax',
    'num_class': 4
}
num_boost_round = 50
bst = xgb.train(params, dtrain, num_boost_round)

cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    nfold=20,
    metrics={'merror'},  # You can use other evaluation metrics
    early_stopping_rounds=10
)


best_num_boost_round = cv_results.shape[0]
# Train the model on the entire dataset
final_model = xgb.train(params, xgb.DMatrix(X, label=y), num_boost_round=best_num_boost_round)
xgb_cv_accuracy = 1 - cv_results['test-merror-mean'].mean()


y_pred = bst.predict(xgb.DMatrix(X))


xgb_mae = mean_absolute_error(y, y_pred)
xgb_mse = mean_squared_error(y, y_pred)
xgb_rmse = np.sqrt(norm_rand_forest_mse)


# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))

C:\Users\calvi\anaconda3\envs\byu-football-play-predictions\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\calvi\anaconda3\envs\byu-football-play-predictions\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [11]:
df = ut.make_big_df()
df = ut.process(df)

X = df.drop(columns=['Play Type']).to_numpy()
y = df['Play Type'].to_numpy()
normalized_x = ut.normalize(X)

X_train, X_test, y_train, y_test = train_test_split(normalized_x, y, test_size=0.2)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

params = {
    'eta': 0.1,
    'objective': 'multi:softmax',
    'num_class': 4
}
num_boost_round = 50
bst = xgb.train(params, dtrain, num_boost_round)

cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    nfold=20,
    metrics={'merror'},  # You can use other evaluation metrics
    early_stopping_rounds=10
)


best_num_boost_round = cv_results.shape[0]
# Train the model on the entire dataset
final_model = xgb.train(params, xgb.DMatrix(X, label=y), num_boost_round=best_num_boost_round)
norm_xgb_cv_accuracy = 1 - cv_results['test-merror-mean'].mean()


y_pred = bst.predict(xgb.DMatrix(X))


norm_xgb_mae = mean_absolute_error(y, y_pred)
norm_xgb_mse = mean_squared_error(y, y_pred)
norm_xgb_rmse = np.sqrt(norm_rand_forest_mse)

C:\Users\calvi\anaconda3\envs\byu-football-play-predictions\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\calvi\anaconda3\envs\byu-football-play-predictions\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


# MLPs

In [ ]:
# load in data
df = ut.make_big_df()
df = ut.process(df)
df = df.dropna().reset_index(drop=True)

X = df.drop(columns=['Play Type']).to_numpy()
y = df['Play Type'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# make model
mlp = MLPClassifier(hidden_layer_sizes=[10], activation='logistic', solver='sgd',
                    alpha=0, batch_size=1, learning_rate_init=.01, shuffle=True,
                    momentum=0, n_iter_no_change=50, max_iter=1000)
# mlp.fit(X_train, y_train)

mlp_cv = cross_val_score(mlp, X, y, cv=5)
print("Cross-validation scores:", mlp_cv)
print("Mean CV accuracy:", mlp_cv.mean())

# Now train on all data
mlp.fit(X, y)

y_pred = mlp.predict(X)
mlp_mae = mean_absolute_error(y, y_pred)
mlp_mse = mean_squared_error(y, y_pred)
mlp_rmse = np.sqrt(mlp_mse)

print(f'mlp_mae: {mlp_mae}')
print(f'mlp_mse: {mlp_mse}')
print(f'mlp_rmse: {mlp_rmse}')

num_itr = mlp.n_iter_

# training set acc
train_pred = mlp.predict(X_train)
# train_acc = accuracy_score(y_train, train_pred)
# train_acc = mlp.score(X_train, y_train)

# test set acc
test_pred = mlp.predict(X_test)
# test_acc = accuracy_score(y_test, test_pred)
test_acc = mlp.score(X_test, y_test)

print("Number of Iterations until Convergence:", num_itr)
# print("Training Set Accuracy:", train_acc)
# print("Test Set Accuracy:", test_acc)

In [ ]:
# make model
mlp = MLPClassifier(hidden_layer_sizes=[10], activation='logistic', solver='sgd',
                    alpha=0, batch_size=1, learning_rate_init=.01, shuffle=True,
                    momentum=0, n_iter_no_change=50, max_iter=1000)
# mlp.fit(X_train, y_train)

normalized_x = ut.normalize(X)
norm_mlp_cv = cross_val_score(mlp, normalized_x, y, cv=5)
print("Cross-validation scores:", mlp_cv)
print("Mean CV accuracy:", norm_mlp_cv.mean())

# Now train on all data
mlp.fit(normalized_x, y)

y_pred = mlp.predict(normalized_x)
norm_mlp_mae = mean_absolute_error(y, y_pred)
norm_mlp_mse = mean_squared_error(y, y_pred)
norm_mlp_rmse = np.sqrt(norm_mlp_mse)

print(f"norm_mlp_mae: {norm_mlp_mae}")
print(f"norm_mlp_mse: {norm_mlp_mse}")
print(f"norm_mlp_rmse: {norm_mlp_rmse}")

In [ ]:
mlp.score(X_train, y_train)

# Tables and Charts and Other Pretty Stuff

In [15]:
from tabulate import tabulate

clf_stats = {
    'Model': 'Random Forest',
    'CV Average Accuracy': round(rand_forest_cv.mean(), 4),
    'MAE': round(rand_forest_mae, 4),
    'MSE': round(rand_forest_mse, 4),
    'RMSE': round(rand_forest_rmse, 4)
}

norm_clf_stats = {
    'Model': 'Random Forest (normalized)',
    'CV Average Accuracy': round(norm_rfst_cv.mean(), 4),
    'MAE': round(norm_rand_forest_mae, 4),
    'MSE': round(norm_rand_forest_mse, 4),
    'RMSE': round(norm_rand_forest_rmse, 4)
}

bst_stats = {
    'Model': 'XGBoost',
    'CV Average Accuracy': round(xgb_cv_accuracy, 4),
    'MAE': round(xgb_mae, 4),
    'MSE': round(xgb_mse, 4),
    'RMSE': round(xgb_rmse, 4)
}

norm_xgb_cv_accuracy
norm_bst_stats = {
    'Model': 'XGBoost (Normalized Inputs)',
    'CV Average Accuracy': round(xgb_cv_accuracy, 4),
    'MAE': round(xgb_mae, 4),
    'MSE': round(xgb_mse, 4),
    'RMSE': round(xgb_rmse, 4)
}

mlp_stats = {
    'Model': 'Multi-layer Perceptron',
    'CV Average Accuracy': round(mlp_cv.mean(), 4),
    'MAE': round(mlp_mae, 4),
    'MSE': round(mlp_mse, 4),
    'RMSE': round(mlp_rmse, 4)
}

norm_mlp_stats = {
    'Model': 'Multi-layer Perceptron (Normalized Inputs)',
    'CV Average Accuracy': round(norm_mlp_cv.mean(), 4),
    'MAE': round(norm_mlp_mae, 4),
    'MSE': round(norm_mlp_mse, 4),
    'RMSE': round(norm_mlp_rmse, 4)
}



# Combine the stats into a list for tabulate
all_stats = [mlp_stats, norm_mlp_stats, clf_stats, norm_clf_stats, bst_stats]

# Create a table
table = tabulate(all_stats, headers="keys", tablefmt="pretty")
print(table)

+----------------------------+---------------------+--------+--------+--------+
|           Model            | CV Average Accuracy |  MAE   |  MSE   |  RMSE  |
+----------------------------+---------------------+--------+--------+--------+
|       Random Forest        |       0.6395        |  0.0   |  0.0   |  0.0   |
| Random Forest (normalized) |        0.642        | 0.0077 | 0.0154 | 0.1242 |
|          XGBoost           |       0.6524        | 0.2793 | 0.5537 | 0.1242 |
+----------------------------+---------------------+--------+--------+--------+
